In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# 选择设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
# data/eng-fra.txt文件：The file is a tab separated list of translation pairs:
# Oh no!	Oh non !
# 每一行是英文句子与对应的法语句子，中间是tab分割
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip()) # 转小写，去头尾空白
    #print(s)
    s = re.sub(r"([.!?])", r" \1", s) # 符号 替换为 空格+符号    
    #print(s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s) # 非a-zA-Z.!?，替换为1个空格 。   意思：只保留预期内的字符，多个空格合并1个空格
    #print(s)
    return s
#print(normalizeString('va !'))
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines] # 英文句子->法语句子
    # print(pairs[0])

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs] # 法语句子->英文句子
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs
#readLangs('eng','fra')

In [4]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

# 只保留英文是上面这种格式开头的样本，丢弃单词数量太多的句子
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [5]:
def prepareData(lang1, lang2, reverse=False):
    # input_lang=fra单词编码集, output_lang=eng单词编码集, pairs=[(法语句子,英文句子),(法语句子,英文句子),....]
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    # 仅保留he is , they are这种样本
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    # 把所有pair过一遍，生成法语和英语各自的word编码
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    # 法语的单词数量
    print(input_lang.name, input_lang.n_words)
    # 英语的单词数量
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs)) # 随机返回一个样本（法语句子,英文句子)

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['elle chante les dernieres chansons populaires .', 'she is singing the latest popular songs .']


In [6]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    # indexs=[2,3,4,5,1], .view之后[[2],[3],[4],[5],[1]]
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# 参数:(法语句子,英语句子)
# 返回:(法语单词列表，英语单词列表)
tensorsFromPair(pairs[0])

(tensor([[2],
         [3],
         [4],
         [5],
         [1]], device='cuda:0'),
 tensor([[2],
         [3],
         [4],
         [1]], device='cuda:0'))

In [7]:
'''
翻译问题不是1个词预测下1个词的简单问题，比如会面临：
1，法语的一个词可能要用2个英语词对应
2，可能法语的2个词对应的英语2个词顺序也不一样
总之翻译问题不是简单的一一映射，
因此需要一个更高级的encoder-decoder模型结构来实现翻译效果.

encoder基于rnn模式将法语句子最终编码成1个向量,作为decode的输入
''' 

'\n翻译问题不是1个词预测下1个词的简单问题，比如会面临：\n1，法语的一个词可能要用2个英语词对应\n2，可能法语的2个词对应的英语2个词顺序也不一样\n总之翻译问题不是简单的一一映射，\n因此需要一个更高级的encoder-decoder模型结构来实现翻译效果.\n\nencoder基于rnn模式将法语句子最终编码成1个向量,作为decode的输入\n'

In [8]:
# encoder把句子编码成词嵌入序列
# decode则根据这个词嵌入序列，从句子开始不断续写下一个单词
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1) # embedded=[1,1,256] 
        output = embedded
        #print('emb',output.shape)
        # GRU模型的输入输出格式：https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
        output, hidden = self.gru(output, hidden)  # output=[1,1,256],hidden=[1,1,256]
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# encoder rnn模型
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)

# 输入：法语句子中某个单词的one-hot向量 + RNN隐层
input_tensor,target_tensor=tensorsFromPair(pairs[0])
print(pairs[0])
print('法语句子的单词数量:',input_tensor.shape)
print(input_tensor[1])
hidden=encoder1.initHidden()

# 前向传播
print('输入:',input_tensor[0].shape,hidden.shape)
output0,hidden=encoder1.forward(input_tensor[0],hidden)
print('输出:',output0.shape,hidden.shape)
output1,hidden=encoder1.forward(input_tensor[1],hidden)
output2,hidden=encoder1.forward(input_tensor[2],hidden)
output3,hidden=encoder1.forward(input_tensor[3],hidden)
output4,hidden=encoder1.forward(input_tensor[4],hidden)
# 1，将输出整理成词嵌入序列，作为decoder的一个输入，用作注意力机制
# 2，将输出的hidden作为decoder的初始hidden
encoder_outputs = torch.zeros(MAX_LENGTH, encoder1.hidden_size, device=device)
encoder_outputs[0]+=output0[0][0]
encoder_outputs[1]+=output1[0][0]
encoder_outputs[2]+=output2[0][0]
encoder_outputs[3]+=output3[0][0]
encoder_outputs[4]+=output4[0][0]
print(encoder_outputs.shape) # 最多能装10个单词的emd向量
print(encoder_outputs) # 少于10个单词的句子会有留下一些全0的emb向量
print(hidden.shape)
print('~~~~~')
# embedding层用法
emb=nn.Embedding(input_lang.n_words, hidden_size, device=device)
emb(input_tensor[0]).shape

['j ai ans .', 'i m .']
法语句子的单词数量: torch.Size([5, 1])
tensor([3], device='cuda:0')
输入: torch.Size([1]) torch.Size([1, 1, 256])
输出: torch.Size([1, 1, 256]) torch.Size([1, 1, 256])
torch.Size([10, 256])
tensor([[-0.1259, -0.2505,  0.3225,  ...,  0.2920,  0.2926, -0.5173],
        [-0.2673, -0.1814,  0.3211,  ...,  0.0372,  0.0379,  0.0874],
        [-0.2593, -0.3610,  0.3063,  ..., -0.1996,  0.2217,  0.0852],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<CopySlices>)
torch.Size([1, 1, 256])
~~~~~


torch.Size([1, 256])

In [9]:
# tensorboard可视化网络结构
from torch.utils.tensorboard import SummaryWriter
with SummaryWriter('runs/encoder') as w:
    hidden_size = 256
    encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    input_tensor,target_tensor=tensorsFromPair(pairs[0])
    hidden=encoder1.initHidden()
    w.add_graph(encoder1, (input_tensor[0],hidden))

In [11]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [12]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [15]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [16]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [18]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [19]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

1m 49s (- 25m 29s) (5000 6%) 2.8542


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)